# [Module 1.3] Horovod 분산 훈련

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 1. 환경 셋업
- 2. 세이지 메이크 로컬 모드 훈련
- 3. SageMaker Host Mode 로 훈련
- 4. 모델 아티펙트 저장

---

### 참고
- [Horovod Official Git](https://github.com/horovod/horovod)
- [Horovod with PyTorch](https://horovod.readthedocs.io/en/stable/pytorch.html)
- [Pytorch Horovod Example on SageMaker ](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-python-sdk/pytorch_horovod_mnist)

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [28]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
### 커스텀 라이브러리
import config 

버킷 및 폴더(prefix) 설정

In [30]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

use_default_bucket = True
if use_default_bucket:
    bucket = sagemaker_session.default_bucket()
else:
    bucket = '<Type your bucket>'
    
prefix = "NCFModel"



# 2. 세이지 메이크 로컬 모드 훈련
#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [31]:
import os
import subprocess


try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


## 2.1. 스크립트 모드의 코드 작성 방법
- ![script_mode_example](img/script_mode_example.png)

## 2.2.훈련 코드 확인
- 아래의 코드는 전형적인 스크립트 모드의 코드 작성 방법을 따르고 있습니다.
- 훈련 함수는 `from train_lib import train` 로서 이전 노트북의 **[세이지 메이커 없이]** 작성한 스크래치 버전에서 사용한 훈련 함수와 동일 합니다.


In [32]:
# train_code = 'src/train_horovod.py'
# !pygmentize {train_code}

## 2.3. 로컬에 있는 데이타 세트의 위치를 지정 합니다.

In [33]:
local_inputs = config.main_path
print("local_inputs: ", local_inputs)

local_inputs:  ../data/


In [34]:
local_inputs = {'train': f'file://{local_inputs}',
          'test': f'file://{local_inputs}'}

## 2.4. 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

In [37]:
hyperparameters = {'epochs': 1, 
                   'lr': 0.001,
                   'batch_size': 256 * 4,
                   'top_k' : 10,
                   'dropout' : 0.0,
                   'factor_num' : 32,
                   'num_layers' : 3,
                   'num_ng' : 4,
                   'test_num_ng' : 99,   
                    }  

In [40]:
from sagemaker.pytorch import PyTorch
import os
import subprocess


local_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
    hyperparameters= hyperparameters               
    
)
local_estimator.fit(local_inputs)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-22-12-55-48-100
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-suf4n:
    command: train
    container_name: n9si5jei9k-algo-1-suf4n
    environment:
    - '[Masked]'
    - '[Masked]'
    image: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.8.1-gpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-suf4n
    runtime: nvidia
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp5b_6pp49/algo-1-suf4n/input:/opt/ml/input
    - /tmp/tmp5b_6pp49/algo-1-suf4n/ou

Creating n9si5jei9k-algo-1-suf4n ... 
Creating n9si5jei9k-algo-1-suf4n ... done
Attaching to n9si5jei9k-algo-1-suf4n
n9si5jei9k-algo-1-suf4n | 2022-05-22 12:55:51,708 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
n9si5jei9k-algo-1-suf4n | 2022-05-22 12:55:51,751 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
n9si5jei9k-algo-1-suf4n | 2022-05-22 12:55:51,754 sagemaker_pytorch_container.training INFO     Invoking user training script.
n9si5jei9k-algo-1-suf4n | 2022-05-22 12:55:51,959 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
n9si5jei9k-algo-1-suf4n | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
n9si5jei9k-algo-1-suf4n | Requirement already satisfied: nvidia-ml-py3==7.352 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (7.352.0)
n9si5jei9k-algo-1-suf4n | Collecting pandas==0.24.2
n9si5jei9k-algo-1-suf4n |   Downloading

# 3. SageMaker Host Mode 로 훈련
- instance_type, session 을 수정 합니다.
- 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
- wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
- 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

## 3.1. 데이터 세트를 S3에 업로드


In [11]:
s3_data_loc = sagemaker_session.upload_data(path=config.main_path, bucket=bucket, 
                                       key_prefix=f"{prefix}/data")
print("s3_data_loc: ", s3_data_loc)

s3_data_loc:  s3://sagemaker-us-east-1-057716757052/NCFModel/data


In [12]:
! aws s3 ls {s3_data_loc} --recursive

2022-05-21 07:03:58     128039 NCFModel/data/.ipynb_checkpoints/ml-1m.test-checkpoint.rating
2022-05-22 11:19:10    2891424 NCFModel/data/ml-1m.test.negative
2022-05-22 11:19:10     128039 NCFModel/data/ml-1m.test.rating
2022-05-22 11:19:10   20982911 NCFModel/data/ml-1m.train.rating
2022-05-21 07:03:58   27404899 NCFModel/data/pinterest-20.test.negative
2022-05-21 07:03:54     807267 NCFModel/data/pinterest-20.test.rating
2022-05-21 07:03:55   21138451 NCFModel/data/pinterest-20.train.rating


## 3.2. 훈련 및 테스트 데이터를 S3 로 지정

In [13]:
s3_inputs = {
            'train': f'{s3_data_loc}',
            'test': f'{s3_data_loc}'
            }

print("s3_inputs: \n", s3_inputs)

s3_inputs: 
 {'train': 's3://sagemaker-us-east-1-057716757052/NCFModel/data', 'test': 's3://sagemaker-us-east-1-057716757052/NCFModel/data'}


## 3.3. 실험 세팅

### 실험(Experiment) 세팅
- Amazon SageMaker 실험은 기계 학습 실험을 구성, 추적, 비교 및 평가할 수 있는 Amazon SageMaker 의 기능입니다
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 실험을 통한 Machine Learning 관리](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/experiments.html)
- sagemaker experiment는 추가적인 패키지를 설치하여야 합니다. 1_Setup_Environment 가 실행이 안되었다고 하면, `!pip install --upgrade sagemaker-experiments` 를 통해 설치 해주세요.
- 여기서는 boto3 API를 통해서 실험을 생성합니다. SageMaker Python SDK를 통해서도 가능합니다.


### 실험 생성

In [14]:
# !pip install --upgrade sagemaker-experiments
import boto3
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from datetime import datetime

sm_client = boto3.client('sagemaker')


# 설험에 대한 이름을 생성 합니다.
experiment_name = prefix + '-single-train'

# 실험이 존재하지 않으면 생성하고, 그렇지 않으면 지나갑니다.
try:
    response = sm_client.describe_experiment(ExperimentName=experiment_name)
    print(f"Experiment:{experiment_name} already exists")    
    
except:
    response = sm_client.create_experiment(
        ExperimentName = experiment_name,
        Description = 'Experiment for NCF',
    )
    print(f"Experiment:{experiment_name} is created")        


Experiment:NCFModel-single-train already exists


### 하이퍼 파라미터 세팅
- epochs 값을 조절해서 실행 시간을 조정 하세요.

In [15]:
host_hyperparameters = {'epochs': 10, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,                   
                    }  

### 시도(Trial) 생성

In [16]:
from datetime import datetime
# 시도 이름 생성
ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
trial_name = experiment_name + f"-{ts}"

# 1개의 실험 안에 시도를 생성함.
response = sm_client.create_trial(
    ExperimentName = experiment_name,
    TrialName = trial_name,
)    

# 실험 설정: 실험 이름, 시도 이름으로 구성
experiment_config = {
    'ExperimentName' : experiment_name,
    'TrialName' : trial_name,
    "TrialComponentDisplayName" : 'Training',
}    



## 3.4 훈련 실행


### 훈련 메트릭을 CloudWatch 에서 보기
- 개발자 가이드
    - [Monitor and Analyze Training Jobs Using Amazon CloudWatch ](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html#define-train-metrics)

In [17]:
metric_definitions=[
       {'Name': 'HR', 'Regex': 'HR=(.*?);'},
       {'Name': 'NDCG', 'Regex': 'NDCG=(.*?);'}
    ]


In [21]:
from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.8xlarge'

host_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters=host_hyperparameters,
    metric_definitions = metric_definitions
    
)
host_estimator.fit(s3_inputs, 
                   experiment_config = experiment_config, # 실험 설정 제공                   
                   wait=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-22-11-20-49-885


In [ ]:
%%time

host_estimator.logs()

2022-05-22 11:20:50 Starting - Starting the training job...ProfilerReport-1653218450: InProgress
...
2022-05-22 11:21:28 Starting - Preparing the instances for training.........
2022-05-22 11:23:13 Downloading - Downloading input data
2022-05-22 11:23:13 Training - Downloading the training image.....................
2022-05-22 11:26:45 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-22 11:26:42,201 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-22 11:26:42,241 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-22 11:26:42,249 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-22 11:26:42,751 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requ

# 4. 실험 결과 보기


위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [ ]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 5
pd.options.display.max_colwidth = 50

search_expression = {
    "Filters": [
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}


trial_component_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
)

trial_component_analytics.dataframe()

### 모델 평가 지표에 순서에 따른 시도 보기
- 아래는 모델 평가 지표에 따른 순서로 보여주기 입니다.

In [ ]:

trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
    sort_by="metrics.HR.max",        
    sort_order="Descending",
    metric_names=["HR"],    
    parameter_names=["epochs", "batch_size", "dropout", "factor_num", 
                     "lr", "num_layers" ,"num_ng", "test_num_ng", "top_k",
                    ],
)

trial_component_training_analytics.dataframe()

# 5. 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [ ]:
horovod_artifact_path = host_estimator.model_data
print("horovod_artifact_path: ", horovod_artifact_path)

%store horovod_artifact_path

기타 변수 저장

In [ ]:
%store bucket 
%store prefix